"""
# Proyek Analisis Data: Bike Sharing Dataset

## 1. Landasan Teori

### Pengertian Analisis Data
Analisis data adalah proses inspeksi, pembersihan, transformasi, dan pemodelan data dengan tujuan menemukan informasi yang berguna, menginformasikan kesimpulan, dan mendukung pengambilan keputusan. Dalam bisnis, ini membantu organisasi beralih dari intuisi semata ke keputusan berbasis fakta.

### Perbedaan Descriptive Analysis dan Exploratory Data Analysis (EDA)
- **Descriptive Analysis**: Fokus pada "apa yang telah terjadi". Ini merangkum data historis menggunakan statistik dasar (mean, median, modus) untuk memberikan gambaran umum.
- **Exploratory Data Analysis (EDA)**: Fokus pada "mengapa ini terjadi" atau "apa yang mungkin terjadi". EDA melibatkan penggalian lebih dalam untuk mencari pola, anomali, menguji hipotesis, dan memeriksa asumsi menggunakan statistik visual dan ringkasan.

### Pentingnya Data Cleaning dan Data Wrangling
Data mentah seringkali kotor (mengandung missing values, duplikat, atau format yang salah).
- **Data Cleaning**: Memperbaiki error dan menghapus data yang tidak akurat.
- **Data Wrangling**: Mengubah format data mentah menjadi format yang lebih siap untuk dianalisis.
Tanpa tahap ini, hasil analisis bisa bias atau tidak valid ("Garbage In, Garbage Out").

### Peran Visualisasi Data
Visualisasi data mengubah angka-angka kompleks menjadi representasi grafis (seperti grafik batang, scatter plot). Ini memudahkan otak manusia untuk mendeteksi tren, pola, dan outlier yang mungkin terlewatkan dalam tabel angka.

### Machine Learning (Opsional)
Pada proyek ini, saya menggunakan **Regresi Linear**.
- **Tujuan**: Memprediksi jumlah penyewaan sepeda (`cnt`) berdasarkan variabel cuaca (suhu, kelembaban, kecepatan angin).
- **Alasan**: Karena target variabel bersifat kontinu (jumlah sewa), regresi adalah pendekatan yang paling tepat untuk melihat hubungan sebab-akibat sederhana.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Data Preparation

In [ ]:
# 1. Memuat Data
try:
    df = pd.read_csv('day.csv')
    print("Data berhasil dimuat!")
except FileNotFoundError:
    print("File 'day.csv' tidak ditemukan.")

In [ ]:
# 2. Menangani Missing Value & Cek Struktur
df.info()
print("\nJumlah Missing Value per kolom:\n")
df.isnull().sum()

In [ ]:
# 3. Transformasi Data
# Mengubah angka menjadi label yang bisa dibaca untuk keperluan visualisasi
# season: 1:Spring, 2:Summer, 3:Fall, 4:Winter
df['season_label'] = df['season'].map({1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'})
# yr: 0:2011, 1:2012
df['year_label'] = df['yr'].map({0: '2011', 1: '2012'})
# weathersit: 1:Clear, 2:Mist, 3:Light Snow/Rain, 4:Heavy Rain
df['weather_label'] = df['weathersit'].map({
    1: 'Clear/Partly Cloudy',
    2: 'Misty/Cloudy',
    3: 'Light Snow/Rain',
    4: 'Severe Weather'
})

# Mengubah kolom tanggal menjadi tipe datetime
df['dteday'] = pd.to_datetime(df['dteday'])

print("\nData setelah transformasi:")
print(df.head())

Exploratory Data Analysis (EDA)

In [ ]:
print("\n--- Statistik Deskriptif ---")
print(df[['cnt', 'temp', 'atemp', 'hum', 'windspeed']].describe())

# Pertanyaan Analisis:
# 1. Bagaimana pengaruh musim (season) terhadap rata-rata jumlah penyewaan sepeda?
# 2. Bagaimana hubungan antara suhu (temp) dengan jumlah penyewaan (cnt)?

# Menjawab Pertanyaan 1: Grouping by Season
seasonal_analysis = df.groupby('season_label')['cnt'].mean().sort_values(ascending=False)
print("\nRata-rata penyewaan berdasarkan musim:\n", seasonal_analysis)

# Menjawab Pertanyaan 2: Korelasi
correlation = df[['temp', 'cnt']].corr()
print("\nKorelasi antara Suhu dan Jumlah Sewa:\n", correlation)

Visualisasi Data

In [ ]:
# Visualisasi 1: Jumlah Penyewaan Sepeda berdasarkan Musim
plt.figure(figsize=(10, 6))
colors = sns.color_palette("viridis", len(seasonal_analysis))
sns.barplot(x=seasonal_analysis.index, y=seasonal_analysis.values)
plt.title('Rata-rata Penyewaan Sepeda per Musim', fontsize=16)
plt.xlabel('Musim', fontsize=12)
plt.ylabel('Rata-rata Jumlah Sewa', fontsize=12)
plt.show()

# Visualisasi 2: Hubungan Suhu vs Jumlah Sewa (Scatter Plot)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='temp', y='cnt', hue='season_label', alpha=0.7)
plt.title('Hubungan Suhu (Normalized) vs Jumlah Penyewaan', fontsize=16)
plt.xlabel('Suhu (Normalized)', fontsize=12)
plt.ylabel('Total Penyewaan', fontsize=12)
plt.legend(title='Musim')
plt.show()

Machine Learning

In [ ]:
print("\n--- Machine Learning: Simple Linear Regression ---")

# Feature Selection
X = df[['temp', 'hum', 'windspeed']] # Fitur: Suhu, Kelembaban, Kecepatan Angin
y = df['cnt'] # Target: Jumlah Sewa

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeling
model = LinearRegression()
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print("Interpretasi: Model regresi ini menjelaskan seberapa besar variasi jumlah sewa yang dapat diprediksi oleh cuaca.")
print("Koefisien (Temp, Hum, Windspeed):", model.coef_)

"""
### Insight & Kesimpulan

**Insight Utama:**
1. **Faktor Musim**: Musim gugur (Fall) memiliki rata-rata penyewaan tertinggi, diikuti oleh musim panas (Summer). Musim semi (Spring) memiliki jumlah penyewaan terendah. Ini menunjukkan orang lebih suka bersepeda saat cuaca hangat namun tidak terlalu ekstrem.
2. **Pengaruh Cuaca**: Terdapat korelasi positif yang kuat antara suhu dan jumlah penyewaan. Semakin tinggi suhu (sampai titik tertentu), semakin banyak orang menyewa sepeda.
3. **Prediksi**: Variabel cuaca (suhu, kelembaban, angin) cukup signifikan dalam memprediksi permintaan sewa sepeda, seperti yang ditunjukkan oleh model regresi sederhana.

**Kesimpulan:**
Penyedia layanan berbagi sepeda sebaiknya memaksimalkan ketersediaan unit pada musim Gugur dan Panas. Strategi promosi bisa difokuskan pada hari-hari dengan prakiraan cuaca cerah dan hangat.
"""